In [84]:
import pandas as pd
import numpy as np

In [85]:
#Import Datasets

#PV Output, change to PVWatts automatic based on inputs
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer load
CustomerLoad = pd.read_excel('https://openei.org/datasets/dataset/399852d9-91a3-4dd7-86ba-73dd5388c994/'
                              'resource/c0608a74-1ef8-4583-a4cd-33f0c78e8478/download/lbnlb74electricity.xlsx')
    #Change to be an excel file: CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

In [86]:
#Input Data

#Storage
    #Fixed
storage_size_kW = float(250)
storage_capex = float(250000)
years_of_use = float(10) #temporary

#Diesel Generator
    #Fixed
diesel_size_kW = float(250)
diesel_capex = float(100000)
diesel_fuel_cost =float(3.5)
diesel_price = float(.40) #temporary

    #Variable
    
#Electricity Price - TOU
    #00:01 - 11:00
    #11:01 - 15:00
    #15:01 - 24:00
    
#Electricity Price - Fixed

electricity_price = float(0.13)

In [87]:
#Clean datasets and convert to hourly

#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']

In [88]:
#Add electricity price, solar production, net customer load

EnergyMaster = CustomerLoad
EnergyMaster['electricity_price'] = electricity_price

#Add pv production
EnergyMaster['PV_Output'] = PVOutput

#post load pv production
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#post PV load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#storage SOC
EnergyMaster['Storage_SOC'] = EnergyMaster['PV_Output_Post_Load'].cumsum().clip(0,storage_size_kW)

#new customer load post storage
EnergyMaster['Storage_SOC_Post_Load'] = (EnergyMaster['Storage_SOC']-EnergyMaster['Customer_Load']).clip_lower(0)

In [89]:
#create randomized grid outages

#variables
days_per_week_grid_down = 3
hoursdown = 4
percent_downtime = days_per_week_grid_down / (7.0*24/hoursdown)

#calculations
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

In [90]:
#calculate costs in scenarios

#diesel scenario

EnergyMaster['cost_in_diesel'] = (diesel_price * EnergyMaster['grid_down'])
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']].
sum(axis=1)
EnergyMaster['diesel_scenario_price'].fillna(EnergyMaster['cost_in_diesel'],inplace=True)

SyntaxError: invalid syntax (<ipython-input-90-178e99211c5a>, line 6)

In [93]:
#storage w PV scenario
years_of_use = EnergyMaster.loc['timestamp_hour'].apply(lambda x: 
pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
EnergyMaster.pivot_table(index=['Customer_Load'], 
                        values='grid_down', aggfunc='sum').reset_index()

#throughput = EnergyMaster.loc[[EnergyMaster['grid_down']<1 and EnergyMaster[timestamp_hour]
#EnergyMaster['cost_in_storageAndpv'] = (storage_capex / years_of_use / throughput)

EnergyMaster.head(50)

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,Storage_SOC,Storage_SOC_Post_Load,grid_down_probability,grid_down
0,22.8125,2014-01-01 00:00:00,0.13,0.000,0.0000,22.8125,0.0000,0.0000,0.659960,0
1,24.6650,2014-01-01 00:00:00,0.13,0.000,0.0000,24.6650,0.0000,0.0000,0.237357,0
2,24.1725,2014-01-01 01:00:00,0.13,0.000,0.0000,24.1725,0.0000,0.0000,0.471438,0
3,24.9500,2014-01-01 01:00:00,0.13,0.000,0.0000,24.9500,0.0000,0.0000,0.474609,0
4,23.6175,2014-01-01 01:00:00,0.13,0.000,0.0000,23.6175,0.0000,0.0000,0.662886,0
5,23.7425,2014-01-01 01:00:00,0.13,0.000,0.0000,23.7425,0.0000,0.0000,0.336418,0
6,24.7725,2014-01-01 02:00:00,0.13,0.000,0.0000,24.7725,0.0000,0.0000,0.750184,0
7,23.4475,2014-01-01 02:00:00,0.13,19.396,0.0000,4.0515,0.0000,0.0000,0.463298,0
8,24.4975,2014-01-01 02:00:00,0.13,230.275,205.7775,0.0000,205.7775,181.2800,0.788220,0
9,24.4175,2014-01-01 02:00:00,0.13,433.377,408.9595,0.0000,250.0000,225.5825,0.608687,0
